##Word Embdding Visualization

## word embeddings on larger corpus

In [ ]:
#generating huge corpus of sentences

In [ ]:
# Generating a corpus of 200 sentences
# For simplicity, we'll create sentences using combinations of some predefined subjects, verbs, and objects.

import random

# Predefined subjects, verbs, and objects
subjects = ["The cat", "A dog", "The bird", "My friend", "The teacher", "A doctor", "The actor", "A scientist"]
verbs = ["saw", "found", "spoke to", "helped", "taught", "loved", "created", "studied"]
objects = ["a book", "the moon", "a car", "the tree", "a computer", "a robot", "a painting", "a bird"]

# Generate sentences
corpus = []
for _ in range(200):
    sentence = f"{random.choice(subjects)} {random.choice(verbs)} {random.choice(objects)}."
    corpus.append(sentence)

# Display the first 10 sentences as a sample
corpus[0:10]



['My friend found a book.',
 'A dog studied a painting.',
 'The bird studied the moon.',
 'A doctor loved a bird.',
 'A doctor studied a painting.',
 'The teacher studied a book.',
 'My friend spoke to a robot.',
 'A scientist found a robot.',
 'A scientist saw the moon.',
 'The bird taught the moon.']

In [ ]:
sentences = [
    "The future king is the prince",
    "Daughter is the princess",
    "Son is the prince",
    "Only a man can be a king",
    "Only a woman can be a queen",
    "The princess will be a queen",
    "Queen and king rule the realm",
    "The prince is a strong man",
    "The princess is a beautiful woman",
    "The royal family is the king and queen and their children",
    "Prince is only a boy now",
    "A boy will be a man",
    "A girl grows up to be a woman",
    "The woman is a mother to the daughter",
    "Every woman was once a girl",
    "The young girl aspires to be a wise woman",
    "The daughter has a close bond with her mother",
    "In every woman is the heart of a girl",
    "The mother love for her daughter is strong",
    "A daughter may inherit her mother traits",
    "The girl and her mother share a special relationship",
    "Womanhood is the journey from a girl to a mature individual",
    "A daughter first role model is often her mother",
    "The girl looked up to her mother, aspiring to be a strong woman"

]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import re

# Text Preprocessing Function
def clean_text(string, punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''', stop_words=['the', 'a', 'and', 'is', 'be', 'will']):
    string = re.sub(r'https?://\S+|www\.\S+', '', string)
    string = re.sub(r'<.*?>', '', string)
    for x in string.lower():
        if x in punctuations:
            string = string.replace(x, "")
    string = string.lower()
    string = ' '.join([word for word in string.split() if word not in stop_words])
    string = re.sub(r'\s+', ' ', string).strip()
    return string

# Creating Context Pairs
def create_context_pairs(texts, window=2):
    word_lists = []
    all_text = []
    for text in texts:
        text = clean_text(text)
        words = text.split()
        all_text += words
        for i, word in enumerate(words):
            for w in range(-window, window + 1):
                if w == 0 or i + w < 0 or i + w >= len(words):
                    continue
                word_lists.append((word, words[i + w]))
    return word_lists, all_text

# Example texts (replace with your corpus)
#texts = ["Your text data goes here.", "Another sentence here."]
texts=sentences
word_lists, all_text = create_context_pairs(texts)

# Creating Unique Word Dictionary
def create_unique_word_dict(text):
    words = list(set(text))
    words.sort()
    return {word: i for i, word in enumerate(words)}


# One-Hot Encoding
def make_one_hot(word_idx, vocab_size):
    x = torch.zeros(vocab_size).float()
    x[word_idx] = 1.0
    return x

unique_word_dict = create_unique_word_dict(all_text)



In [ ]:
unique_word_dict
#Creating one Hot Encoding
make_one_hot(3,len(unique_word_dict))

tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.])

In [ ]:
unique_word_dict

{'aspires': 0,
 'aspiring': 1,
 'beautiful': 2,
 'bond': 3,
 'boy': 4,
 'can': 5,
 'children': 6,
 'close': 7,
 'daughter': 8,
 'every': 9,
 'family': 10,
 'first': 11,
 'for': 12,
 'from': 13,
 'future': 14,
 'girl': 15,
 'grows': 16,
 'has': 17,
 'heart': 18,
 'her': 19,
 'in': 20,
 'individual': 21,
 'inherit': 22,
 'journey': 23,
 'king': 24,
 'looked': 25,
 'love': 26,
 'man': 27,
 'mature': 28,
 'may': 29,
 'model': 30,
 'mother': 31,
 'now': 32,
 'of': 33,
 'often': 34,
 'once': 35,
 'only': 36,
 'prince': 37,
 'princess': 38,
 'queen': 39,
 'realm': 40,
 'relationship': 41,
 'role': 42,
 'royal': 43,
 'rule': 44,
 'share': 45,
 'son': 46,
 'special': 47,
 'strong': 48,
 'their': 49,
 'to': 50,
 'traits': 51,
 'up': 52,
 'was': 53,
 'wise': 54,
 'with': 55,
 'woman': 56,
 'womanhood': 57,
 'young': 58}

In [ ]:
# Neural Network Definition
class SimpleNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SimpleNN, self).__init__()
        self.linear1 = nn.Linear(vocab_size, embedding_dim)
        self.linear2 = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        hidden = self.linear1(x)
        out = self.linear2(hidden)
        return out

class EnhancedNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(EnhancedNN, self).__init__()
        self.linear1 = nn.Linear(vocab_size, embedding_dim * 2)
        self.linear2 = nn.Linear(embedding_dim * 2, embedding_dim * 2)
        self.linear3 = nn.Linear(embedding_dim * 2, embedding_dim)
        self.linear4 = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = torch.relu(self.linear2(x))
        x = torch.relu(self.linear3(x))
        out = self.linear4(x)
        return out

# Model Initialization
vocab_size = len(unique_word_dict)
embedding_dim = 2  # For visualization
#model = SimpleNN(vocab_size, embedding_dim)
model=EnhancedNN(vocab_size, embedding_dim)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Using Adam optimizer

# Prepare Data for Batch Training
def make_context_vector(context, word_to_ix):
    idx = [word_to_ix[w] for w in context]
    return torch.tensor(idx, dtype=torch.long)

# Create dataset for DataLoader
X_train = []
Y_train = []
for context, target in word_lists:
    context_idx = unique_word_dict[context]
    target_idx = unique_word_dict[target]
    X_train.append(make_one_hot(context_idx, vocab_size))
    Y_train.append(torch.tensor(target_idx, dtype=torch.long))

X_train = torch.stack(X_train)
Y_train = torch.stack(Y_train)
train_data = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_data, shuffle=True, batch_size=256)

# Training Loop
for epoch in range(4000):
    total_loss = 0
    for context, target in train_loader:
        model.zero_grad()
        log_probs = model(context)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}, Loss: {total_loss}")




Epoch 0, Loss: 8.271615028381348
Epoch 1, Loss: 8.298410415649414
Epoch 2, Loss: 8.264976978302002
Epoch 3, Loss: 8.220711708068848
Epoch 4, Loss: 8.209547519683838
Epoch 5, Loss: 8.305041313171387
Epoch 6, Loss: 8.227733135223389
Epoch 7, Loss: 8.253612041473389
Epoch 8, Loss: 8.27634572982788
Epoch 9, Loss: 8.196739673614502
Epoch 10, Loss: 8.208773612976074
Epoch 11, Loss: 8.284508228302002
Epoch 12, Loss: 8.21584939956665
Epoch 13, Loss: 8.212746620178223
Epoch 14, Loss: 8.227931022644043
Epoch 15, Loss: 8.265592575073242
Epoch 16, Loss: 8.261763572692871
Epoch 17, Loss: 8.257358074188232
Epoch 18, Loss: 8.204750537872314
Epoch 19, Loss: 8.154468536376953
Epoch 20, Loss: 8.182283401489258
Epoch 21, Loss: 8.260927677154541
Epoch 22, Loss: 8.206844806671143
Epoch 23, Loss: 8.148524284362793
Epoch 24, Loss: 8.227122783660889
Epoch 25, Loss: 8.166606426239014
Epoch 26, Loss: 8.182592391967773
Epoch 27, Loss: 8.258803844451904
Epoch 28, Loss: 8.241822719573975
Epoch 29, Loss: 8.17298936

In [ ]:
weights

array([[-1.5574738 ,  0.6508319 , -0.28739223,  0.39145634, -1.209842  ,
        -0.5494101 ,  1.0032015 ,  0.43033925,  0.29976258, -0.6062208 ,
         0.566927  , -0.98628074, -0.92631024,  0.7913819 ,  1.051587  ,
         0.98769987, -0.21085939,  0.9649723 ,  0.8433093 ,  0.46126434,
        -0.3002649 ],
       [ 1.1278356 , -1.1454614 , -0.37823954,  1.2937189 ,  1.0112084 ,
        -0.01094009, -0.758142  ,  0.84450656, -0.7785674 , -1.1798913 ,
        -0.04335337, -1.0542378 ,  1.0882869 ,  1.0705504 ,  0.40984106,
         0.22181965,  0.6825908 , -0.77056724, -0.7812412 ,  0.2530708 ,
         0.5268758 ]], dtype=float32)

In [ ]:
# Extracting Word Embeddings
weights = model.linear1.weight.detach().numpy().T  # Transpose the weights matrix

# Check the shape of transposed weights
print(f"Shape of transposed weights: {weights.shape}")

# Visualization
plt.figure(figsize=(20, 20))
for word, idx in unique_word_dict.items():
    if idx < weights.shape[0]:  # Ensure the index is within bounds
        plt.scatter(weights[idx, 0], weights[idx, 1])
        plt.annotate(word, (weights[idx, 0], weights[idx, 1]), fontsize=8)
plt.show()


##PyTorch Embedding Layer

In [ ]:
#Initialize Embdding Layer

import torch.nn as nn

embedding = nn.Embedding(num_embeddings=10000, embedding_dim=300)


In [ ]:
#using Embedding Layers

import torch

# Create an embedding object
embedding = nn.Embedding(10, 3)  # 10 words in vocab, 3-dimensional embeddings

# Indices of words
input = torch.LongTensor([1, 2, 8, 4])

# Get embeddings for input indices
output = embedding(input)
print(output)


tensor([[ 1.4609, -1.0005,  2.0904],
        [-0.8883,  0.8422, -0.3980],
        [ 1.0307,  0.6627,  0.9245],
        [ 1.6249,  0.6349, -0.5316]], grad_fn=<EmbeddingBackward0>)


Using Embedding Layers in Neural Network

In [ ]:
class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # Additional layers here

    def forward(self, x):
        x = self.embedding(x)
        # Pass through additional layers
        return x


Comparing Two Sentences using pytorch word embeddings

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Example sentences
sentence1 = "The quick brown fox"
sentence2 = "The lazy dog"

# Tokenize sentences (naively by splitting on spaces, in practice use a proper tokenizer)
tokens1 = sentence1.lower().split()
tokens2 = sentence2.lower().split()

# Assume a vocabulary mapping tokens to indices
vocab = {"the": 0, "quick": 1, "brown": 2, "fox": 3, "lazy": 4, "dog": 5}
vocab_size = len(vocab)

# Map tokens to indices
indices1 = [vocab[token] for token in tokens1 if token in vocab]
indices2 = [vocab[token] for token in tokens2 if token in vocab]

# Embedding layer
embedding_dim = 50
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# Convert indices to tensors
indices_tensor1 = torch.tensor(indices1, dtype=torch.long)
indices_tensor2 = torch.tensor(indices2, dtype=torch.long)

# Get embeddings
embeddings1 = embedding(indices_tensor1)
embeddings2 = embedding(indices_tensor2)

# Aggregate embeddings (e.g., by averaging)
sentence_embedding1 = torch.mean(embeddings1, dim=0)
sentence_embedding2 = torch.mean(embeddings2, dim=0)

# Compare embeddings (e.g., using cosine similarity)
cosine_sim = F.cosine_similarity(sentence_embedding1.unsqueeze(0), sentence_embedding2.unsqueeze(0))

print("Cosine Similarity:", cosine_sim.item())


Cosine Similarity: 0.2784857153892517


Loading Pretrained word2vec Embeddings

In [ ]:
!pip install gensim


In [ ]:
from gensim.models import KeyedVectors

# Load Word2Vec model (this might take some time)
word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)


In [ ]:
import torch
import numpy as np

vocab_size = len(word2vec_model.vocab)
embedding_dim = word2vec_model.vector_size

# Create an embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Fill the embedding matrix with the vectors from Word2Vec
for i, word in enumerate(word2vec_model.vocab):
    embedding_vector = word2vec_model[word]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [ ]:
import torch.nn as nn

# Define the embedding layer
embedding_layer = nn.Embedding(vocab_size, embedding_dim)
embedding_layer.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))

# Freeze the embeddings (optional but recommended)
embedding_layer.weight.requires_grad = False


In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.embedding = embedding_layer
        # Other layers of the model...

    def forward(self, x):
        x = self.embedding(x)
        # Forward pass through the rest of the model
        return x

# Example usage
model = MyModel()


##Question Answering using LSTM

In [ ]:
import csv
import torch
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import torch.nn as nn
import torch.optim as optim



In [ ]:
def load_trivia_qa_data(file_path, max_length):
    questions, answers = [], []
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:

          q= row[1]
          a=row[3]
          if len(a.split())<100:

            q_tokens = q.lower().split()[:max_length]
            a_tokens = a.lower().split()[:max_length]
            questions.append(q_tokens)
            answers.append(a_tokens)
    return questions, answers

def create_vocab(texts):
    vocab = set()
    for text in texts:
        vocab.update(text)
    return {word: i+1 for i, word in enumerate(vocab)}, {i+1: word for i, word in enumerate(vocab)}

def texts_to_sequences(texts, vocab_to_int, max_length):
    sequences = []
    for text in texts:
        seq = [vocab_to_int.get(word, 0) for word in text]
        seq += [0] * (max_length - len(seq))  # Padding
        sequences.append(seq)
    return sequences

max_length = 100  # Maximum length of question/answer
questions, answers = load_trivia_qa_data('/content/train.csv', max_length)
vocab_to_int, int_to_vocab = create_vocab(questions + answers)
questions_seq = texts_to_sequences(questions, vocab_to_int, max_length)
answers_seq = texts_to_sequences(answers, vocab_to_int, max_length)


In [ ]:
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on device: {device}")


Training on device: cuda


In [ ]:
class QADataset(Dataset):
    def __init__(self, questions, answers):
        self.questions = questions
        self.answers = answers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, index):
        return torch.tensor(self.questions[index]), torch.tensor(self.answers[index])

dataset = QADataset(questions_seq, answers_seq)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        output, hidden = self.lstm(x, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        # Initialize hidden states to GPU, if available
        return (torch.zeros(n_layers, batch_size, hidden_dim).to(device),
                torch.zeros(n_layers, batch_size, hidden_dim).to(device))

# Hyperparameters
vocab_size = len(vocab_to_int) + 1  # +1 for padding token
embedding_dim = 256
hidden_dim = 512
n_layers = 2

#model = LSTMModel(vocab_size, embedding_dim, hidden_dim, n_layers)
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, n_layers).to(device)


In [ ]:

def train(model, dataloader, epochs, lr):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()

    for epoch in range(epochs):
        for questions, answers in dataloader:
            # Initialize hidden state with the current batch size
            batch_size = questions.size(0)
            hidden = model.init_hidden(batch_size)

            # Move data to GPU if available
            questions, answers = questions.to(device), answers.to(device)
            hidden = tuple([each.data for each in hidden])

            optimizer.zero_grad()
            output, hidden = model(questions, hidden)
            loss = criterion(output.transpose(1, 2), answers)
            loss.backward()
            optimizer.step()

        print(f'Epoch: {epoch+1}, Loss: {loss.item()}')


batch_size = 64  # or another batch size that fits your GPU
train(model, dataloader, epochs=10, lr=0.001)



Epoch: 1, Loss: 1.580116629600525
Epoch: 2, Loss: 1.488576054573059
Epoch: 3, Loss: 1.2370301485061646
Epoch: 4, Loss: 2.697777032852173
Epoch: 5, Loss: 0.9038660526275635
Epoch: 6, Loss: 3.0431902408599854
Epoch: 7, Loss: 1.5565505027770996
Epoch: 8, Loss: 2.2447376251220703
Epoch: 9, Loss: 2.121448278427124
Epoch: 10, Loss: 1.667191743850708


In [ ]:
def predict(model, question_seq, vocab_to_int, int_to_vocab, max_length):
    model.eval()
    question_seq = torch.tensor([vocab_to_int.get(word, 0) for word in question_seq.lower().split()][:max_length])
    question_seq = question_seq.unsqueeze(0).to(device)  # Add batch dimension and send to GPU
    hidden = model.init_hidden(1)

    output, hidden = model(question_seq, hidden)
    answer_ids = output.argmax(dim=2).cpu()  # Move the results back to CPU for processing
    return ' '.join([int_to_vocab[i.item()] for i in answer_ids[0]])

# Example prediction
question = "what continent is australia"
answer = predict(model, question, vocab_to_int, int_to_vocab, max_length)
print(answer)



the is is is


##LSTM text Generation

In [ ]:
import requests

# Downloading "Pride and Prejudice" by Jane Austen from Project Gutenberg
url = "https://www.gutenberg.org/files/1342/1342-0.txt"
response = requests.get(url)
text = response.text

# Displaying the first 500 characters to confirm download
text[:500]


'ï»¿The Project Gutenberg eBook of Pride and prejudice, by Jane Austen\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located befo'

In [ ]:
!wget "https://www.gutenberg.org/files/1342/1342-0.txt"

--2024-02-06 01:46:23--  https://www.gutenberg.org/files/1342/1342-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 772186 (754K) [text/plain]
Saving to: ‘1342-0.txt’

1342-0.txt          100%[===================>] 754.09K  2.47MB/s    in 0.3s    

2024-02-06 01:46:24 (2.47 MB/s) - ‘1342-0.txt’ saved [772186/772186]



In [ ]:
import torch
import torch.nn as nn
import string
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# Read and preprocess the text
file_path = '/content/1342-0.txt'  # Change to your file path
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read().lower()
text = "".join([c for c in text if c not in string.punctuation])

# Creating character dictionaries
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

In [ ]:
# Encode the text
encoded = np.array([char2int[ch] for ch in text])

# Function to create batches
def create_batches(arr, batch_size, seq_length):
    total_batch_size = batch_size * seq_length
    n_batches = len(arr) // total_batch_size
    arr = arr[:n_batches * total_batch_size]
    arr = arr.reshape((batch_size, -1))

    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [ ]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Defining the LSTM model
class TextGeneratorLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers):
        super(TextGeneratorLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        lstm_out, hidden = self.lstm(embedded, hidden)
        out = self.fc(lstm_out)
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden


In [ ]:
# Initializing the model
vocab_size = len(chars)
embedding_dim = 400
hidden_dim = 512
n_layers = 2
# Initialize the model and move it to GPU if available
model = TextGeneratorLSTM(vocab_size, embedding_dim, hidden_dim, n_layers).to(device)


In [ ]:
# Modifying the train function to handle GPU operations
def train(model, data, epochs=10, batch_size=10, seq_length=50, lr=0.001):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        hidden = model.init_hidden(batch_size)

        for x, y in create_batches(data, batch_size, seq_length):
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            hidden = tuple([each.data for each in hidden])

            model.zero_grad()
            output, hidden = model(x, hidden)
            loss = criterion(output.transpose(1, 2), y)
            loss.backward()
            optimizer.step()

        print(f'Epoch: {epoch+1}/{epochs}... Loss: {loss.item()}')


In [ ]:
# Function to generate new text (with modifications for GPU)
def generate_text(model, size, prime='The', top_k=5):
    model.eval()
    model.to(device)

    chars = [ch for ch in prime]
    h = model.init_hidden(1)
    for ch in prime:
        char, h = predict(model, ch, h, top_k=top_k, device=device)

    chars.append(char)

    for _ in range(size):
        char, h = predict(model, chars[-1], h, top_k=top_k, device=device)
        chars.append(char)

    return ''.join(chars)

In [ ]:
def predict(model, char, h=None, top_k=None, device='cpu'):
    x = np.array([[char2int[char]]])
    x = torch.from_numpy(x).to(device)

    h = tuple([each.data for each in h])
    out, h = model(x, h)

    p = torch.nn.functional.softmax(out, dim=2).data
    p = p.cpu()  # Move to CPU for numpy operations

    if top_k is not None:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.cpu().numpy().squeeze()  # Move to CPU for numpy operations
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
    else:
        char = np.argmax(p).numpy()

    return int2char[char], h

In [ ]:
# Running the training
train(model, encoded, epochs=20, batch_size=128, seq_length=100)



Epoch: 1/20... Loss: 1.9876174926757812
Epoch: 2/20... Loss: 1.5944464206695557
Epoch: 3/20... Loss: 1.416271686553955
Epoch: 4/20... Loss: 1.3206251859664917
Epoch: 5/20... Loss: 1.2621331214904785
Epoch: 6/20... Loss: 1.2183165550231934
Epoch: 7/20... Loss: 1.1869114637374878
Epoch: 8/20... Loss: 1.1608729362487793
Epoch: 9/20... Loss: 1.133046269416809
Epoch: 10/20... Loss: 1.109960913658142
Epoch: 11/20... Loss: 1.093722939491272
Epoch: 12/20... Loss: 1.0758144855499268
Epoch: 13/20... Loss: 1.0492336750030518
Epoch: 14/20... Loss: 1.0368629693984985
Epoch: 15/20... Loss: 1.0282034873962402
Epoch: 16/20... Loss: 1.007260799407959
Epoch: 17/20... Loss: 1.0004738569259644
Epoch: 18/20... Loss: 0.976285994052887
Epoch: 19/20... Loss: 0.9668840169906616
Epoch: 20/20... Loss: 0.9565847516059875


KeyError: 'I'

In [ ]:

# Generate some text with lowercase prime string
print(generate_text(model, 100, prime='it is a truth', top_k=5))


it is a truth of this work on the
other whether he would not have a week owed one of these
andwards of the present


##Captions generator Sequence to Sequence

##Pycoco dataset download


In [ ]:
!pip install pycocotools


In [ ]:
#downloading complete dataset
import os
import requests
from tqdm import tqdm

# Function to download file with progress bar
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in tqdm(r.iter_content(chunk_size=chunk_size), desc='Downloading'):
            fd.write(chunk)

# Define URLs for the COCO dataset (2017 Training images and annotations as example)
image_zip_url = 'http://images.cocodataset.org/zips/train2017.zip'
annotation_zip_url = 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'

# Define save paths
save_dir = './coco'
os.makedirs(save_dir, exist_ok=True)
image_zip_path = os.path.join(save_dir, 'train2017.zip')
annotation_zip_path = os.path.join(save_dir, 'annotations_trainval2017.zip')

# Download images and annotations
download_url(image_zip_url, image_zip_path)
download_url(annotation_zip_url, annotation_zip_path)

# Extract the downloaded files
import zipfile

with zipfile.ZipFile(image_zip_path, 'r') as zip_ref:
    zip_ref.extractall(save_dir)

with zipfile.ZipFile(annotation_zip_path, 'r') as zip_ref:
    zip_ref.extractall(save_dir)

print('Download and extraction completed.')


In [ ]:
!rm -rf "/content/YOUR_PATH"

In [ ]:
#downloading only 3000 images and captions since complete dataset is too huge

from pycocotools.coco import COCO
import requests
from tqdm import tqdm
import os
import json

def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in tqdm(r.iter_content(chunk_size=chunk_size), desc='Downloading'):
            fd.write(chunk)

# Initialize paths and URLs
dataDir = 'YOUR_PATH/coco'
dataType = 'train2017'
annotations_url = 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
annotations_zip_path = os.path.join(dataDir, 'annotations_trainval2017.zip')
annFile = f'{dataDir}/annotations/captions_{dataType}.json'

# Ensure the directory exists
os.makedirs(os.path.dirname(annFile), exist_ok=True)

# Download and extract the annotations file if not already present
if not os.path.exists(annFile):
    download_url(annotations_url, annotations_zip_path)
    import zipfile
    with zipfile.ZipFile(annotations_zip_path, 'r') as zip_ref:
        zip_ref.extractall(dataDir)
    os.remove(annotations_zip_path)  # Clean up zip file

# Initialize COCO API for caption annotations
coco = COCO(annFile)

# Get all image ids, then slice the list to get only the first 5000
imgIds = coco.getImgIds()
imgIds = imgIds[:500]

# Filter annotations for the selected images
annIds = coco.getAnnIds(imgIds=imgIds)
annotations = coco.loadAnns(annIds)

# Create a new annotations dict with filtered images and annotations
filtered_data = {
    "images": [coco.loadImgs(id)[0] for id in imgIds],
    "annotations": annotations,
    "type": "captions",
    "info": coco.dataset['info'],
    "licenses": coco.dataset['licenses']

}

# Save the filtered annotations to a new file
filtered_annFile = os.path.join(dataDir, f'filtered_captions_{dataType}.json')
with open(filtered_annFile, 'w') as f:
    json.dump(filtered_data, f)

print(f"Filtered annotations saved to {filtered_annFile}")

# Download the specified number of images
base_url = 'http://images.cocodataset.org/train2017/'
for img in tqdm(filtered_data["images"], desc="Downloading Images"):
    img_url = base_url + img['file_name']
    save_path = os.path.join(dataDir, dataType, img['file_name'])
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    download_url(img_url, save_path)

print('Download of 5000 images and corresponding annotations completed.')



In [ ]:
!pip install pycocotools


In [ ]:
!pip list

##Step 1: Import Libraries

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d aladdinpersson/flickr8kimagescaptions

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!rm -rf flickr8k

In [ ]:
!kaggle datasets download -d aladdinpersson/flickr8kimagescaptions

100% 1.03G/1.04G [00:30<00:00, 39.9MB/s]
100% 1.04G/1.04G [00:30<00:00, 36.9MB/s]


In [ ]:
!unzip /content/flickr8kimagescaptions.zip -d flickr8k

In [ ]:
import os

# Define the paths
image_dir = 'flickr8k/flickr8k/images'
captions_file_path = 'flickr8k/flickr8k/captions.txt'
cleaned_captions_file_path = 'flickr8k/cleaned_captions.txt'

# Read captions and check image existence
with open(captions_file_path, 'r') as captions_file, open(cleaned_captions_file_path, 'w') as cleaned_captions_file:
    for line in captions_file:
        image_file_name = line.split(',')[0]  # Adjusted to split by comma
        if os.path.exists(os.path.join(image_dir, image_file_name)):
            cleaned_captions_file.write(line)
        else:
            print(f"Image {image_file_name} not found, skipping.")


Image image not found, skipping.


In [ ]:
import os  # when loading file paths
import pandas as pd  # for lookup in annotation file
import spacy  # for tokenizer
import torch
from torch.nn.utils.rnn import pad_sequence  # pad batch
from torch.utils.data import DataLoader, Dataset
from PIL import Image  # Load img
import torchvision.transforms as transforms


# We want to convert text -> numerical values
# 1. We need a Vocabulary mapping each word to a index
# 2. We need to setup a Pytorch dataset to load the data
# 3. Setup padding of every batch (all examples should be
#    of same seq_len and setup dataloader)
# Note that loading the image is very easy compared to the text!

# Download with: python -m spacy download en
spacy_eng = spacy.load("en_core_web_sm")


class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1

                else:
                    frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]


class FlickrDataset(Dataset):
    def __init__(self, root_dir, captions_file, transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(captions_file)
        self.transform = transform

        # Get img, caption columns
        self.imgs = self.df["image"]
        self.captions = self.df["caption"]

        # Initialize vocabulary and build vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]

        # Check if the file exists
        if not os.path.exists(os.path.join(self.root_dir, img_id)):
            # Handle the missing file (e.g., by skipping, using a placeholder, etc.)
            print(f"Warning: File {os.path.join(self.root_dir, img_id)} not found.")
            return None # or handle differently
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])

        return img, torch.tensor(numericalized_caption)


class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_idx)

        return imgs, targets


def get_loader(
    root_folder,
    annotation_file,
    transform,
    batch_size=32,
    num_workers=8,
    shuffle=True,
    pin_memory=True,
):
    dataset = FlickrDataset(root_folder, annotation_file, transform=transform)

    pad_idx = dataset.vocab.stoi["<PAD>"]

    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory,
        collate_fn=MyCollate(pad_idx=pad_idx),
    )

    return loader, dataset


In [ ]:
transform = transforms.Compose(
        [transforms.Resize((224, 224)), transforms.ToTensor(),]
    )

loader, dataset = get_loader(
    "/content/flickr8k/flickr8k/images/", "/content/flickr8k/flickr8k/captions.txt", transform=transform
)

"""
for idx, (imgs, captions) in enumerate(loader):
    print(imgs.shape)
    print(captions.shape)
"""

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


'\nfor idx, (imgs, captions) in enumerate(loader):\n    print(imgs.shape)\n    print(captions.shape)\n'

In [ ]:
import torch
import torch.nn as nn
import statistics
import torchvision.models as models


class EncoderCNN(nn.Module):
    def __init__(self, embed_size, train_CNN=False):
        super(EncoderCNN, self).__init__()
        self.train_CNN = train_CNN
        # Enable auxiliary logits
        self.inception = models.inception_v3(pretrained=True, aux_logits=True)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
        self.relu = nn.ReLU()
        self.times = []
        self.dropout = nn.Dropout(0.5)
    """
    def forward(self, images):
        # Handle both main and auxiliary outputs
        features, aux_outputs = self.inception(images)
        return self.dropout(self.relu(features)), aux_outputs
    """
    def forward(self, images):
      # Assuming the encoder might return a tuple when aux_logits=True, but we ignore auxiliary outputs here
      features = self.inception(images)
      if isinstance(features, tuple):  # This check is for models that return a tuple
          features = features[0]  # Use only the main output
      return features


class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, features, captions):
        embeddings = self.dropout(self.embed(captions))
        embeddings = torch.cat((features.unsqueeze(0), embeddings), dim=0)
        hiddens, _ = self.lstm(embeddings)
        outputs = self.linear(hiddens)
        return outputs


class CNNtoRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(CNNtoRNN, self).__init__()
        self.encoderCNN = EncoderCNN(embed_size)
        self.decoderRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

    def forward(self, images, captions):
        features = self.encoderCNN(images)
        outputs = self.decoderRNN(features, captions)
        return outputs

    def caption_image(self, image, vocabulary, max_length=50):
        result_caption = []

        with torch.no_grad():
            x = self.encoderCNN(image).unsqueeze(0)
            states = None

            for _ in range(max_length):
                hiddens, states = self.decoderRNN.lstm(x, states)
                output = self.decoderRNN.linear(hiddens.squeeze(0))
                predicted = output.argmax(1)
                result_caption.append(predicted.item())
                x = self.decoderRNN.embed(predicted).unsqueeze(0)

                if vocabulary.itos[predicted.item()] == "<EOS>":
                    break

        return [vocabulary.itos[idx] for idx in result_caption]

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image


def print_examples(model, device, dataset):
    transform = transforms.Compose(
        [
            transforms.Resize((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

    model.eval()
    test_img1 = transform(Image.open("test_examples/dog.jpg").convert("RGB")).unsqueeze(
        0
    )
    print("Example 1 CORRECT: Dog on a beach by the ocean")
    print(
        "Example 1 OUTPUT: "
        + " ".join(model.caption_image(test_img1.to(device), dataset.vocab))
    )
    test_img2 = transform(
        Image.open("test_examples/child.jpg").convert("RGB")
    ).unsqueeze(0)
    print("Example 2 CORRECT: Child holding red frisbee outdoors")
    print(
        "Example 2 OUTPUT: "
        + " ".join(model.caption_image(test_img2.to(device), dataset.vocab))
    )
    test_img3 = transform(Image.open("test_examples/bus.png").convert("RGB")).unsqueeze(
        0
    )
    print("Example 3 CORRECT: Bus driving by parked cars")
    print(
        "Example 3 OUTPUT: "
        + " ".join(model.caption_image(test_img3.to(device), dataset.vocab))
    )
    test_img4 = transform(
        Image.open("test_examples/boat.png").convert("RGB")
    ).unsqueeze(0)
    print("Example 4 CORRECT: A small boat in the ocean")
    print(
        "Example 4 OUTPUT: "
        + " ".join(model.caption_image(test_img4.to(device), dataset.vocab))
    )
    test_img5 = transform(
        Image.open("test_examples/horse.png").convert("RGB")
    ).unsqueeze(0)
    print("Example 5 CORRECT: A cowboy riding a horse in the desert")
    print(
        "Example 5 OUTPUT: "
        + " ".join(model.caption_image(test_img5.to(device), dataset.vocab))
    )
    model.train()


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    step = checkpoint["step"]
    return step

In [ ]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter



def train():
    transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

    train_loader, dataset = get_loader(
        root_folder="/content/flickr8k/flickr8k/images",
        annotation_file="/content/flickr8k/flickr8k/captions.txt",
        transform=transform,
        num_workers=2,
    )

    torch.backends.cudnn.benchmark = True
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    load_model = False
    save_model = False
    train_CNN = False

    # Hyperparameters
    embed_size = 256
    hidden_size = 256
    vocab_size = len(dataset.vocab)
    num_layers = 1
    learning_rate = 3e-4
    num_epochs = 100

    # for tensorboard
    writer = SummaryWriter("runs/flickr")
    step = 0

    # initialize model, loss etc
    model = CNNtoRNN(embed_size, hidden_size, vocab_size, num_layers).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Only finetune the CNN
    for name, param in model.encoderCNN.inception.named_parameters():
        if "fc.weight" in name or "fc.bias" in name:
            param.requires_grad = True
        else:
            param.requires_grad = train_CNN

    if load_model:
        step = load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)

    model.train()

    for epoch in range(num_epochs):
        # Uncomment the line below to see a couple of test cases
        # print_examples(model, device, dataset)

        if save_model:
            checkpoint = {
                "state_dict": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "step": step,
            }
            save_checkpoint(checkpoint)

        for idx, (imgs, captions) in tqdm(
            enumerate(train_loader), total=len(train_loader), leave=False
        ):
            imgs = imgs.to(device)
            captions = captions.to(device)

            outputs = model(imgs, captions[:-1])
            loss = criterion(
                outputs.reshape(-1, outputs.shape[2]), captions.reshape(-1)
            )

            writer.add_scalar("Training loss", loss.item(), global_step=step)
            step += 1

            optimizer.zero_grad()
            loss.backward(loss)
            optimizer.step()

In [ ]:
train()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 159MB/s] 
  7%|▋         | 87/1265 [00:25<04:58,  3.94it/s]

In [ ]:
device

##Attention Mechanism Practical Example

In [ ]:
import torch
import torch.nn.functional as F

# Define queries, keys, and values
queries = torch.rand((2, 4))  # (num_queries, embedding_dim)
keys = torch.rand((3, 4))     # (num_keys, embedding_dim)
values = torch.rand((3, 4))   # (num_values, embedding_dim)

# Calculate attention scores
scores = torch.matmul(queries, keys.T)  # (num_queries, num_keys)

# Apply softmax to get attention weights
attention_weights = F.softmax(scores, dim=-1)

# Compute the weighted sum of values
attended_values = torch.matmul(attention_weights, values)  # (num_queries, embedding_dim)

# Print the output
print("Attention Weights:", attention_weights)
print("Attended Values:", attended_values)


Attention Weights: tensor([[0.2436, 0.4661, 0.2903],
        [0.2957, 0.4441, 0.2603]])
Attended Values: tensor([[0.5787, 0.1710, 0.5884, 0.3941],
        [0.5922, 0.1747, 0.6116, 0.4012]])


##Integrating Attention Mechanism with LSTM for Sequence Classification

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


In [ ]:
class LSTMWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(LSTMWithAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.attention = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # lstm_out: [batch_size, seq_length, hidden_dim]

        # Apply attention
        attention_weights = F.softmax(self.attention(lstm_out).squeeze(2), dim=1)
        # Multiply attention weights with LSTM outputs (element-wise)
        attended = torch.mul(lstm_out, attention_weights.unsqueeze(2).expand_as(lstm_out))
        # Sum over the sequence dimension
        representation = torch.sum(attended, dim=1)

        output = self.fc(representation)
        return output, attention_weights


In [ ]:
def generate_data(num_samples, seq_length, input_dim):
    np.random.seed(42)
    X = np.random.rand(num_samples, seq_length, input_dim)
    y = np.sum(X, axis=(1,2)) > (seq_length * input_dim / 2)  # Arbitrary classification rule
    y = y.astype(int)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

# Generate synthetic data
num_samples = 1000
seq_length = 10
input_dim = 1
X, y = generate_data(num_samples, seq_length, input_dim)
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
X

In [ ]:
model = LSTMWithAttention(input_dim, hidden_dim=50, output_dim=2, n_layers=1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):  # Loop over the dataset multiple times
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs, attention_weights = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 0.6773175597190857
Epoch 2, Loss: 0.678956151008606
Epoch 3, Loss: 0.5369415879249573
Epoch 4, Loss: 0.30714279413223267
Epoch 5, Loss: 0.7640738487243652
Epoch 6, Loss: 0.21662567555904388
Epoch 7, Loss: 0.3214204013347626
Epoch 8, Loss: 0.15604577958583832
Epoch 9, Loss: 0.12088260799646378
Epoch 10, Loss: 0.09580276161432266


In [ ]:
# Generate synthetic test data
num_test_samples = 300  # Number of test samples
X_test, y_test = generate_data(num_test_samples, seq_length, input_dim)
test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Inference mode, gradients not needed
        for inputs, labels in dataloader:
            outputs, _ = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model
accuracy = evaluate_model(model, test_dataloader)
print(f'Accuracy on test data: {accuracy:.2f}%')


Accuracy on test data: 91.67%
